# Scrittopoli

In [20]:
%matplotlib inline

# Import the necessary packages and modules
import matplotlib
matplotlib.style.use('ggplot')
import matplotlib.pyplot as plt
import matplotlib.dates  as mpdt
import matplotlib.ticker as mptk

import numpy as np
import pytz
mytz=pytz.timezone("Europe/Rome")
import pandas as pd
import datetime as dt
import math
import importlib
import random
import itertools
import csv


## Definizione dei gironi della fase eliminatoria

Per la determinazione delle teste di serie sono considerate le classifiche:
* classifica generale di MI 2017;
* classifiche di girone di MI 100;
* classifiche di tappa di Ferragosto d'Inchiostro 2017.

Non è considerata la classifica del Contest di Halloween 2017 perché non era un contest per racconti.

Regole per assegnare i punteggi nelle classifiche aggiuntive:
* primo classificato: 5 punti
* secondo classificato: 3 punti
* terzo classificato: 2 punti
* quarto classificato: 1 punto

I gironi saranno da 4 squadre.


In [2]:
MAX_PER_GIRONE=4

## Manca l'ultimo MI
mi2017_csv="data/classifica_mi2017.csv"

mi100_12_csv="data/classifica_mi100_1-2.csv"
mi100_34_csv="data/classifica_mi100_3-4.csv"
mi100_56_csv="data/classifica_mi100_5-6.csv"
mi100_78_csv="data/classifica_mi100_7-8.csv"

fdi2017_1_csv="data/fdi2017_1.csv"
fdi2017_2_csv="data/fdi2017_2.csv"
fdi2017_3_csv="data/fdi2017_3.csv"

## Valido solo dal 14 dicembre
squadre_csv="data/squadre.csv"


In [3]:
mi2017=pd.read_csv(mi2017_csv,sep=":",quotechar='"',header=0,index_col=1)

mi100_12=pd.read_csv(mi100_12_csv,sep=",",quotechar='"',header=0,index_col=1)
mi100_12=mi100_12.fillna(0)
mi100_34=pd.read_csv(mi100_34_csv,sep=",",quotechar='"',header=0,index_col=1)
mi100_34=mi100_34.fillna(0)
mi100_56=pd.read_csv(mi100_56_csv,sep=",",quotechar='"',header=0,index_col=1)
mi100_56=mi100_56.fillna(0)
mi100_78=pd.read_csv(mi100_78_csv,sep=",",quotechar='"',header=0,index_col=1)
mi100_78=mi100_78.fillna(0)

fdi2017_1=pd.read_csv(fdi2017_1_csv,sep=",",quotechar='"',header=0,index_col=0)
fdi2017_1=fdi2017_1.fillna(0)
fdi2017_2=pd.read_csv(fdi2017_2_csv,sep=",",quotechar='"',header=0,index_col=0)
fdi2017_2=fdi2017_2.fillna(0)
fdi2017_3=pd.read_csv(fdi2017_3_csv,sep=",",quotechar='"',header=0,index_col=0)
fdi2017_3=fdi2017_3.fillna(0)

squadre=pd.read_csv(squadre_csv,sep=":",quotechar='"',header=0,index_col=1)


### Ranking

In [4]:
tabellone=mi2017.copy()

tabellone=tabellone[["punteggio"]]
tabellone=tabellone.rename(index=str,columns={"punteggio":"mi2017"})

fdi2017_1_v=fdi2017_1[["titolo","voti"]].sort_values(by="voti",ascending=False)[:4]
fdi2017_2_v=fdi2017_2[["titolo","voti"]].sort_values(by="voti",ascending=False)[:5]
fdi2017_3_v=fdi2017_3[["titolo","voti"]].sort_values(by="voti",ascending=False)[:4]

fdi2017_1_v["fdi2017_1"]=[4,4,1.5,1.5]
fdi2017_2_v["fdi2017_2"]=[5,2.5,2.5,.5,.5]
fdi2017_3_v["fdi2017_3"]=[5,3,2,1]

tabellone=tabellone.merge(fdi2017_1_v[["fdi2017_1"]],
                          left_index=True,right_index=True,how="outer")
tabellone=tabellone.merge(fdi2017_2_v[["fdi2017_2"]],
                          left_index=True,right_index=True,how="outer")
tabellone=tabellone.merge(fdi2017_3_v[["fdi2017_3"]],
                          left_index=True,right_index=True,how="outer")

mi100_12_v=mi100_12[["titolo","voti"]].sort_values(by="voti",ascending=False)[:5]
mi100_34_v=mi100_34[["titolo","voti"]].sort_values(by="voti",ascending=False)[:4]
mi100_56_v=mi100_56[["titolo","voti"]].sort_values(by="voti",ascending=False)[:5]
mi100_78_v=mi100_78[["titolo","voti"]].sort_values(by="voti",ascending=False)[:4]

mi100_12_v["mi100_1"]=[5,3,2,.5,.5]
mi100_34_v["mi100_2"]=[5,3,2,1]
mi100_56_v["mi100_3"]=[4,4,1,1,1]
mi100_78_v["mi100_4"]=[5,3,1.5,1.5]

tabellone=tabellone.merge(mi100_12_v.groupby("autore")[["mi100_1"]].sum(),
                          left_index=True,right_index=True,how="outer")
tabellone=tabellone.merge(mi100_34_v.groupby("autore")[["mi100_2"]].sum(),
                          left_index=True,right_index=True,how="outer")
tabellone=tabellone.merge(mi100_56_v.groupby("autore")[["mi100_3"]].sum(),
                          left_index=True,right_index=True,how="outer")
tabellone=tabellone.merge(mi100_78_v.groupby("autore")[["mi100_4"]].sum(),
                          left_index=True,right_index=True,how="outer")



In [5]:
tabellone=tabellone.fillna(0)
tabellone["totale"]=tabellone["mi2017"]
tabellone["totale"]+=tabellone["fdi2017_1"]
tabellone["totale"]+=tabellone["fdi2017_2"]
tabellone["totale"]+=tabellone["fdi2017_3"]
tabellone["totale"]+=tabellone["mi100_1"]
tabellone["totale"]+=tabellone["mi100_2"]
tabellone["totale"]+=tabellone["mi100_3"]
tabellone["totale"]+=tabellone["mi100_4"]

tabellone.sort_values(by="totale",ascending=False)


,mi2017,fdi2017_1,fdi2017_2,fdi2017_3,mi100_1,mi100_2,mi100_3,mi100_4,totale
Macleo,18.33,4.0,0.5,0.0,5.0,1.0,0.0,1.5,30.33
Bango Skank,16.50,0.0,0.0,0.0,0.0,0.0,0.0,5.0,21.50
Rica,15.00,1.5,2.5,0.0,0.0,0.0,0.0,0.0,19.00
Unius,12.66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.66
simone volponi,5.00,4.0,0.0,3.0,0.0,0.0,0.0,0.0,12.00
Marcello,7.00,0.0,0.0,0.0,0.0,0.0,0.0,3.0,10.00
Vincenzo Iennaco,4.33,0.0,0.0,5.0,0.0,0.0,0.0,0.0,9.33
Lizz,3.00,0.0,0.0,0.0,0.5,3.0,0.0,1.5,8.00
Cristina,3.33,0.0,0.0,0.0,0.0,0.0,4.0,0.0,7.33
Joyopi,5.00,0.0,0.0,2.0,0.0,0.0,0.0,0.0,7.00


In [6]:
scrittopoli=tabellone.copy()
scrittopoli=scrittopoli.merge(squadre,left_index=True,right_index=True,how="right").fillna(0)

scrittopoli=scrittopoli.reset_index().set_index(["squadra","giocatore"])

ranking=scrittopoli.groupby("squadra").sum().sort_values(by="totale",ascending=False)[["totale"]]
ranking.reset_index(inplace=True)

ranking

,squadra,totale
0,Living Stones,39.66
1,Green House,34.50
2,D Euforiche,20.50
3,I Bananari,12.00
4,La threesome,9.83
5,Retroguardia Trash,7.00
6,Le TreMende,4.50
7,Gli scrittori del quartierino,1.33
8,Armata Brancaleone,0.00
9,the 3 BrOthers,0.00


### Suddivisione in gironi

In base al numero massimo di squadre in un girone (MAX_PER_GIRONE), viene stabilito il numero dei gironi (num_gironi).

In [7]:
num_squadre=ranking["squadra"].size
num_gironi=int(math.ceil(num_squadre/MAX_PER_GIRONE))
print("Massimo numero di squadre per girone:",MAX_PER_GIRONE)
print("                   Numero di squadre:",num_squadre)
print("                    Numero di gironi:",num_gironi)

Massimo numero di squadre per girone: 4
                   Numero di squadre: 10
                    Numero di gironi: 3


Le squadre vengono suddivise in 4 fasce in base al ranking.

In [8]:
f=1
fasce=[]
for n in range(num_squadre):
    fasce.append("F%d" % f)
    if not (n+1)%num_gironi: f+=1

ranking["fascia"]=fasce

if num_squadre%num_gironi:
    for n in range(num_gironi-num_squadre%num_gironi):
        df2 = pd.DataFrame([["(riposo)",0.0,"F4"]], columns=["squadra","totale","fascia"])
        ranking=ranking.append(df2,ignore_index=True)
    
ranking

,squadra,totale,fascia
0,Living Stones,39.66,F1
1,Green House,34.50,F1
2,D Euforiche,20.50,F1
3,I Bananari,12.00,F2
4,La threesome,9.83,F2
5,Retroguardia Trash,7.00,F2
6,Le TreMende,4.50,F3
7,Gli scrittori del quartierino,1.33,F3
8,Armata Brancaleone,0.00,F3
9,the 3 BrOthers,0.00,F4


In [9]:
gironi_data=[]

for f in ["F1","F2","F3","F4"]:
    f_list=list(ranking[ranking["fascia"]==f]["squadra"])
    random.shuffle(f_list)
    gironi_data.append(f_list)

gironi_labels=[]
for n in range(num_gironi):
    gironi_labels.append(chr(65+n))

gironi=pd.DataFrame(gironi_data,columns=gironi_labels)
gironi

,A,B,C
0,Green House,D Euforiche,Living Stones
1,Retroguardia Trash,La threesome,I Bananari
2,Armata Brancaleone,Gli scrittori del quartierino,Le TreMende
3,(riposo),the 3 BrOthers,(riposo)


## Calendario

In [10]:

def calendario(label):
    pivot=gironi[label][0]
    others=list(gironi[label][1:])
    ret=[]
    for n in range(3):
        ret.append( [g,n+1,1,pivot,others[n]] )
        ret.append( [g,n+1,2]+others[:n]+others[n+1:] )
    return ret

calendario_data=[]
for g in gironi_labels:
    calendario_data+=calendario(g)


calendario=pd.DataFrame(calendario_data,columns=["girone","giornata","partita",
                                                 "squadra 1","squadra 2"])

cal_idx=["giornata","girone","partita"]

calendario.sort_values(by=cal_idx).set_index(cal_idx)

squadra 1  \
giornata girone partita                                  
1        A      1                          Green House   
                2                   Armata Brancaleone   
         B      1                          D Euforiche   
                2        Gli scrittori del quartierino   
         C      1                        Living Stones   
                2                          Le TreMende   
2        A      1                          Green House   
                2                   Retroguardia Trash   
         B      1                          D Euforiche   
                2                         La threesome   
         C      1                        Living Stones   
                2                           I Bananari   
3        A      1                          Green House   
                2                   Retroguardia Trash   
         B      1                          D Euforiche   
                2                         La threesome   
         C      1                        Living Stones   
                2                           I Bananari   

                                             squadra 2  
giornata girone partita                                 
1        A      1                   Retroguardia Trash  
                2                             (riposo)  
         B      1                         La threesome  
                2                       the 3 BrOthers  
         C      1                           I Bananari  
                2                             (riposo)  
2        A      1                   Armata Brancaleone  
                2                             (riposo)  
         B      1        Gli scrittori del quartierino  
                2                       the 3 BrOthers  
         C      1                          Le TreMende  
                2                             (riposo)  
3        A      1                             (riposo)  
                2                   Armata Brancaleone  
         B      1                       the 3 BrOthers  
                2        Gli scrittori del quartierino  
         C      1                             (riposo)  
                2                          Le TreMende

In [22]:
label=d.strftime("%Y%m%d_%H%M%S")
calendario_csv="data/calendario_"+label+".csv"
with open(calendario_csv, 'w', newline='') as csvfile:
    w = csv.writer(csvfile,delimiter=':',
                   quotechar='"', quoting=csv.QUOTE_MINIMAL)
    w.writerow(["gironi","giornata","partita","squadra 1","squadra 2"])
    for row in calendario_data:
        w.writerow(row)
        
label=d.strftime("%Y%m%d_%H%M%S")
gironi_csv="data/gironi_"+label+".csv"
with open(gironi_csv, 'w', newline='') as csvfile:
    w = csv.writer(csvfile,delimiter=':',
                   quotechar='"', quoting=csv.QUOTE_MINIMAL)
    w.writerow(gironi_labels)
    for row in gironi_data:
        w.writerow(row)

calendario_data

[['A', 1, 1, 'Green House', 'Retroguardia Trash'],
 ['A', 1, 2, 'Armata Brancaleone', '(riposo)'],
 ['A', 2, 1, 'Green House', 'Armata Brancaleone'],
 ['A', 2, 2, 'Retroguardia Trash', '(riposo)'],
 ['A', 3, 1, 'Green House', '(riposo)'],
 ['A', 3, 2, 'Retroguardia Trash', 'Armata Brancaleone'],
 ['B', 1, 1, 'D Euforiche', 'La threesome'],
 ['B', 1, 2, 'Gli scrittori del quartierino', 'the 3 BrOthers'],
 ['B', 2, 1, 'D Euforiche', 'Gli scrittori del quartierino'],
 ['B', 2, 2, 'La threesome', 'the 3 BrOthers'],
 ['B', 3, 1, 'D Euforiche', 'the 3 BrOthers'],
 ['B', 3, 2, 'La threesome', 'Gli scrittori del quartierino'],
 ['C', 1, 1, 'Living Stones', 'I Bananari'],
 ['C', 1, 2, 'Le TreMende', '(riposo)'],
 ['C', 2, 1, 'Living Stones', 'Le TreMende'],
 ['C', 2, 2, 'I Bananari', '(riposo)'],
 ['C', 3, 1, 'Living Stones', '(riposo)'],
 ['C', 3, 2, 'I Bananari', 'Le TreMende']]

In [12]:
gironi_data

[['Green House', 'D Euforiche', 'Living Stones'],
 ['Retroguardia Trash', 'La threesome', 'I Bananari'],
 ['Armata Brancaleone', 'Gli scrittori del quartierino', 'Le TreMende'],
 ['(riposo)', 'the 3 BrOthers', '(riposo)']]

'20171206_163321'